<a href="https://colab.research.google.com/github/ibrahim-syah/indonesian-distilbert-finetuned-squad/blob/main/notebook/Squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2021 Rifky Bujana Bisri & Muhammad Fajrin Buyang Daffa

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

Copyright 2023 Ibrahim Syah Qardhawi

Made some changes to make it easier to look at the plot and change the pretrained model to Indonesian DistilBERT base.

idk anything about licensing chatgpt told me to write it like this whatever

In [ ]:
!pip install datasets evaluate
!pip install transformers -U
!pip install accelerate -U
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !git config --global user.email "ibrahim.syah.q@gmail.com"
# !git config --global user.name "ibrahim-syah"

In [ ]:
import transformers

transformers.__version__

In [ ]:
impossible_answer = True
model_checkpoint = "cahya/distilbert-base-indonesian"
batch_size = 16

# Runtime Settings

In [ ]:
# !nvidia-smi

# Data Preparation

## Load Dataset

In [ ]:
import json

#  # if using the qa-ind2 dataset in kaggle
dev_filename = "/kaggle/input/qa-ind2/dev-v2.0.json"
train_filename = "/kaggle/input/qa-ind2/train-v2.0.json"

# # otherwise, download them yourself
# !wget https://storage.depia.wiki/squad/tar/dev-v2.0.json
# !wget https://storage.depia.wiki/squad/tar/train-v2.0.json
# dev_filename = "dev-v2.0.json"
# train_filename = "train-v2.0.json"

# google colab default /content folder
# dev_filename = "/content/drive/MyDrive/dev-v2.0.json"
# train_filename = "/content/drive/MyDrive/train-v2.0.json"

with open(dev_filename) as f:
    dev = json.load(f)

with open(train_filename) as f:
    train = json.load(f)

🤗 does provide the [squad_v2](https://huggingface.co/datasets/squad_v2) dataset that has been formatted for training but that dataset was for the original SQuAD2.0 whereas the dataset we will use is the [Translated SQuAD2.0](https://github.com/Wikidepia/indonesian_datasets/tree/master/question-answering/squad).

In [ ]:
print("There are %s data in this split"%len(train["data"])) # number of different data titles

i = 3
title = train["data"][i]["title"]
paragraphs = train["data"][i]["paragraphs"]
print("The title for the data indexed at %s is %s"%(i, title))
print("Said data title has %s paragraph(s)"%(len(paragraphs)))
print("The first three paragraph is the following: ")
paragraphs[:3]

It's always a good idea to take a look at our data to get a feel of what they represent. If you compare this format with the finished one from 🤗 squad_v2 you will see that 🤗 squad_v2 pair together the question and answer along with the corresponding id, title, and context while removing everything else. Try looking at the dev split and see if you notice any difference ;).

Since the this raw dataset has a different format than what was expected by our training model (we'll use 🤗 AutoModelForQuestionAnswering)  we will need to format them so each pair of question and answer is an individual data.

In [ ]:
def format(content):
    hf_data = []
    for data in content["data"]:
        title = data["title"]
        for paragraph in data["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                fill = {
                    "id":  qa["id"],
                    "title": title,
                    "context": context,
                    "question": qa["question"],
                    "answers": {"answer_start": [], "text": []}
                }
                if qa["is_impossible"]:
                    answers = qa["plausible_answers"]
                else:
                    answers = qa["answers"]
                for answer in answers:
                    fill["answers"]["answer_start"].append(answer["answer_start"])
                    fill["answers"]["text"].append(answer["text"])

                hf_data.append(fill)

    return hf_data

In [ ]:
%%time
dev = format(dev)
train = format(train)

In [ ]:
# # use small dataset for testing
# train = train[:1000]
# dev = dev[:100]
# len(train), len(dev)

In [ ]:
from datasets import DatasetDict, Dataset
import pandas as pd

datasets = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train)),
    'validation': Dataset.from_pandas(pd.DataFrame(dev))
})
datasets

In [ ]:
datasets['train']['id'][0]

## Data Analyzation

In [ ]:
import random
import numpy as np


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame((np.asarray(dev)[picks]).tolist())

    df['answer_start'] = [i['answer_start'] for i in df['answers']]
    df['answer_text'] = [i['text'] for i in df['answers']]

    del df['answers']

    return df

We are creating this dataframe to make them easier to work with in data analyzation. For training, we will still be using the original 'datasets' DatasetDict.

In [ ]:
from IPython.display import display, HTML

display(HTML(show_random_elements(dev).to_html()))

In [ ]:
train_df = pd.DataFrame(train)

train_df['answer_start'] = [i['answer_start'] for i in train_df['answers']]
train_df['answer_text'] = [i['text'] for i in train_df['answers']]

del train_df['answers']

In [ ]:
dev_df = pd.DataFrame(dev)

dev_df['answer_start'] = [i['answer_start'] for i in dev_df['answers']]
dev_df['answer_text'] = [i['text'] for i in dev_df['answers']]

del dev_df['answers']

Let's see how the length of each context, question, and answer for both of these splits.

In [ ]:
import matplotlib.pyplot as plt

# Set up the figure and axes
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Define the datasets and their corresponding titles
plot_datasets = [(train_df, "Training Dataset"), (dev_df, "Validation Dataset")]

# Define the column names and corresponding titles
columns = [('context', "Context length histogram"),
           ('question', "Question length histogram"),
           ('answer_text', "Answer length histogram")]

# Loop through datasets and columns to plot histograms
for i, (dataset, dataset_title) in enumerate(plot_datasets):
    for j, (column, title) in enumerate(columns):
        ax = axes[i, j]
        if column == "answer_text":
            pd.DataFrame([len(n[0]) for n in dataset['answer_text']]).plot.hist(title=title, bins=20, ax=ax, grid=True)
        else:
            dataset[column].apply(len).plot.hist(title=title, bins=20, ax=ax, grid=True)
        ax.set_xlabel("Length")
        ax.set_ylabel("Frequency")
        ax.grid(True)
        if j == 0:
            ax.set_ylabel("Frequency")
        if i == 1:
            ax.set_xlabel("Length")
        if i == 0:
            ax.set_title(title)
#         if j == 0:
#             ax.set_title(dataset_title)

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()

It seems that the contexts, questions, and answers within the validation set (bottom) are shorter than their training counterpart, or maybe that's just because they lack more data.

## Text Preprocessing

The 🤗 [docs](https://huggingface.co/learn/nlp-course/chapter7/7?fw=pt#processing-the-training-data) has an excellent piece on this particular preprocessing part. In short, we will split long contexts into multiple features. Here is an example:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting is needed.

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

In [ ]:
for x in tokenized_example["input_ids"][:]:
    print(tokenizer.decode(x))

Compared to the original:

In [ ]:
example

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][1][:100])

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Because of the feature split, we need to be careful when looking if the answer is within the context fully or not. We can use sequence_ids() to help us.

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

Let's see an example where we tokenize four examples with max_length of 100 and a stride of 50 so it will produce more features (more likely for a feature to not contain our answer).

In [ ]:
inputs = tokenizer(
    datasets["train"][2:6]["question"],
    datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

In [ ]:
answers = datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

Which means we didn't find the answer in the 0th and the 5th feature so we just denote that with the (0, 0) tuple or [CLS] token.

In [ ]:
idx = 1
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

In [ ]:
idx = 5
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])
decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")
print("decoded example: ",decoded_example)

Now we can start preprocessing the training set.

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace with strip()
    questions = [q.strip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = inputs.pop("offset_mapping")
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0) or [CLS]
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_datasets = datasets.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=datasets["train"].column_names,
)
tokenized_datasets

Now we can get to fine-tuning

## Fine Tuning Model

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoModel, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# model_checkpoint = "boimbukanbaim/distilbert-indonesian-squad"
# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "distilbert-indonesian-squad",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
#     push_to_hub=True,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
# trainer.save_model("distilbert-indonesian-squad")
# trainer.push_to_hub(commit_message="Training complete")

## Simple Evaluation (EM & F1)

Before we get to the evaluation, we need to consider the post-processing. The model will output logits for the start and end positions of the answer in the input IDs. A usual 🤗 pipeline will compute the score by softmaxing the logits and taking the product to each start-end pair and then pick the best valid answer.

We don't need to compute actual scores to make training faster. So we don't need to softmax and taking the product of the start-end pair. We also won't take all the scores but only 20 best scores. 

The way we take the score from the pair is just by taking the sum. We can do this because we skipped the softmax so our output is still logits and
$\log{(ab)} = \log{(a)} + \log{(b)}$

Let's demonstrate this by making a prediction from a subset of our validation dataset with a trained model. The model that we will use is a finetuned distilbert from 🤗. We can preprocess said dataset with the tokenizer provided by the trained model.

However, before that we need to tweak the preprocessing for this validation dataset to add an `example_id` feature so that we can map the newly created features to the original context.

In contrast to the ```preprocess_training_examples```, We can use the validation set to interpret the model's prediction into spans of the original contex.

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        # collect feature(s) belonging to the same id
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        # turn all non-context token to None
        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)
validation_dataset

Slightly different from the dataset that we use in training but this one can be used for our simple evaluation. Now we can get to the evaluation. First we will see how well the original distilbert model by 🤗 performs on this validation dataset.

In [ ]:
small_eval_set = datasets["validation"].select(range(100)) # only 100 because I keep getting out of memory error
trained_checkpoint = "distilbert-base-cased-distilled-squad"

# since preprocess_validation_examples uses tokenizer namespace
# we will change the tokenizer temporarily
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)
eval_set

Let's immediately change the tokenizer variable back to the one we're using before we forgot.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

We can then remove some unnecessary columns (for feeding them into the model) and just feed our data in batches if using GPU.

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

batch

In [ ]:
with torch.no_grad():
    outputs = trained_model(**batch)

We'll change the output into numpy arrays.

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

start_logits.shape, end_logits.shape

It seems like from the original 100 subset from the validation set, it has turned into 124 features each with 384 output logits of start-end pairs. Next up is mapping the predicted answer for each of these feature to their corresponding example.

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)
    
example_to_features

We'll use a simple loop to go through each example with their associated features. We'll then look at the 20 best logit scores that are valid. An invalid prediction would be ones that aren't inside their context, an answer with negative length, or an answer that's too long.

After which we can pick the one with the best score.

In [ ]:
n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

We formatted ``predicted_answer`` with the format that is expected for the metric provided by 🤗 Evaluate library. Let's see if it performs well with the original squad metric.

In [ ]:
import evaluate

metric = evaluate.load("squad")

This particular `squad` metric expects the predicted answer in the format of list of dict with one key for example ID and one key for the predicted text while the format of the theoretical answer in the format below:

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [ ]:
print(predicted_answers[7])
print(theoretical_answers[7])

And here's the metric of all the small subset of validated set:

In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

This model (distilbert-base-cased-distilled-squad by 🤗) was not trained on Indonesian dataset, hence why our scores are poor. Let's put everything in compute_metrics().

Normally, this function will be included in the ``Trainer`` object and will be used during the training loop. But here we'll just make a simple function that we can use at the end of training to see how well our model performs.

In [ ]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

Let's see if our model [distilbert-indonesian-squad](https://huggingface.co/boimbukanbaim/distilbert-indonesian-squad) would do the trick:

In [ ]:
small_eval_set = datasets["validation"].select(range(100))
# small_eval_set = datasets["validation"]
trained_checkpoint = "boimbukanbaim/distilbert-indonesian-squad" # our trained model

# since preprocess_validation_examples uses tokenizer namespace
# we will change the tokenizer temporarily
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)
    
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

Much better! what if we use the metric for SQuAD2.0? We will have to change the formatting to this:

Predictions : List of triple for question-answers to score with the following key-value pairs:

* 'id': the question-answer identification field of the question and answer pair
* 'prediction_text' : the text of the answer
* 'no_answer_probability' : the probability that the question has no answer

References: List of question-answers dictionaries with the following key-value pairs:

* 'id': id of the question-answer pair (see above),
* 'answers': a list of Dict {'text': text of the answer as a string}
* 'no_answer_threshold': the probability threshold to decide that a question has no answer.

## SQuADv2.0 Evaluation (WIP)

In [ ]:
# import torch

# for batch in trainer.get_eval_dataloader():
#     break
# batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
# with torch.no_grad():
#     output = trainer.model(**batch)
# output.keys()

In [ ]:
# output.start_logits.shape, output.end_logits.shape

In [ ]:
# output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

In [ ]:
# n_best_size = 20

In [ ]:
# import numpy as np

# start_logits = output.start_logits[0].cpu().numpy()
# end_logits = output.end_logits[0].cpu().numpy()
# # Gather the indices the best start/end logits:
# start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
# end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
# valid_answers = []
# for start_index in start_indexes:
#     for end_index in end_indexes:
#         if start_index <= end_index: # We need to refine that test to check the answer is inside the context
#             valid_answers.append(
#                 {
#                     "score": start_logits[start_index] + end_logits[end_index],
#                     "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
#                 }
#             )

In [ ]:
# pad_on_right = True

In [ ]:
# def prepare_validation_features(examples):
#     # Some of the questions have lots of whitespace on the left, which is not useful and will make the
#     # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
#     # left whitespace
#     examples["question"] = [q.lstrip() for q in examples["question"]]

#     # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
#     # in one example possible giving several features when a context is long, each of those features having a
#     # context that overlaps a bit the context of the previous feature.
#     tokenized_examples = tokenizer(
#         examples["question" if pad_on_right else "context"],
#         examples["context" if pad_on_right else "question"],
#         truncation="only_second" if pad_on_right else "only_first",
#         max_length=max_length,
#         stride=doc_stride,
#         return_overflowing_tokens=True,
#         return_offsets_mapping=True,
#         padding="max_length",
#     )

#     # Since one example might give us several features if it has a long context, we need a map from a feature to
#     # its corresponding example. This key gives us just that.
#     sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

#     # We keep the example_id that gave us this feature and we will store the offset mappings.
#     tokenized_examples["example_id"] = []

#     for i in range(len(tokenized_examples["input_ids"])):
#         # Grab the sequence corresponding to that example (to know what is the context and what is the question).
#         sequence_ids = tokenized_examples.sequence_ids(i)
#         context_index = 1 if pad_on_right else 0

#         # One example can give several spans, this is the index of the example containing this span of text.
#         sample_index = sample_mapping[i]
#         tokenized_examples["example_id"].append(examples["id"][sample_index])

#         # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
#         # position is part of the context or not.
#         tokenized_examples["offset_mapping"][i] = [
#             (o if sequence_ids[k] == context_index else None)
#             for k, o in enumerate(tokenized_examples["offset_mapping"][i])
#         ]

#     return tokenized_examples


In [ ]:
# small_eval_set = datasets["validation"].select(range(100))

# validation_features = small_eval_set.map(
#     prepare_validation_features,
#     batched=True,
#     remove_columns=datasets["validation"].column_names
# )

# validation_features

In [ ]:
# import os
# torch.cuda.empty_cache() 

In [ ]:
# from transformers import AutoModelForQuestionAnswering, AutoModel, TrainingArguments, Trainer

# trained_checkpoint = "boimbukanbaim/distilbert-indonesian-squad"
# model = AutoModel.from_pretrained(trained_checkpoint)

In [ ]:
# from transformers import TrainingArguments, Trainer

# batch_size = 1

# args = TrainingArguments(
#     "distilbert-indonesian-squad",
#     evaluation_strategy = "epoch",
#     save_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     report_to="none",
# #     push_to_hub=True,
# )

# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

In [ ]:
# # fix the out of memory error
# raw_predictions = trainer.predict(validation_features)

In [ ]:
# validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
# max_answer_length = 30

In [ ]:
# start_logits = output.start_logits[0].cpu().numpy()
# end_logits = output.end_logits[0].cpu().numpy()
# offset_mapping = validation_features[0]["offset_mapping"]
# # The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# # an example index
# context = datasets["validation"][0]["context"]

# # Gather the indices the best start/end logits:
# start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
# end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
# valid_answers = []
# for start_index in start_indexes:
#     for end_index in end_indexes:
#         # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
#         # to part of the input_ids that are not in the context.
#         if (
#             start_index >= len(offset_mapping)
#             or end_index >= len(offset_mapping)
#             or offset_mapping[start_index] is None
#             or offset_mapping[end_index] is None
#         ):
#             continue
#         # Don't consider answers with a length that is either < 0 or > max_answer_length.
#         if end_index < start_index or end_index - start_index + 1 > max_answer_length:
#             continue
#         if start_index <= end_index: # We need to refine that test to check the answer is inside the context
#             start_char = offset_mapping[start_index][0]
#             end_char = offset_mapping[end_index][1]
#             valid_answers.append(
#                 {
#                     "score": start_logits[start_index] + end_logits[end_index],
#                     "text": context[start_char: end_char]
#                 }
#             )

# valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
# valid_answers

In [ ]:
# valid_answers[5]

In [ ]:
# datasets["validation"][1231]

In [ ]:
# import collections

# examples = datasets["validation"]
# features = validation_features

# example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
# features_per_example = collections.defaultdict(list)
# for i, feature in enumerate(features):
#     features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
# from tqdm.auto import tqdm

# def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
#     all_start_logits, all_end_logits = raw_predictions
#     # Build a map example to its corresponding features.
#     example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
#     features_per_example = collections.defaultdict(list)
#     for i, feature in enumerate(features):
#         features_per_example[example_id_to_index[feature["example_id"]]].append(i)

#     # The dictionaries we have to fill.
#     predictions = collections.OrderedDict()

#     # Logging.
#     print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

#     # Let's loop over all the examples!
#     for example_index, example in enumerate(tqdm(examples)):
#         # Those are the indices of the features associated to the current example.
#         feature_indices = features_per_example[example_index]

#         min_null_score = None # Only used if squad_v2 is True.
#         valid_answers = []

#         context = example["context"]
#         # Looping through all the features associated to the current example.
#         for feature_index in feature_indices:
#             # We grab the predictions of the model for this feature.
#             start_logits = all_start_logits[feature_index]
#             end_logits = all_end_logits[feature_index]
#             # This is what will allow us to map some the positions in our logits to span of texts in the original
#             # context.
#             offset_mapping = features[feature_index]["offset_mapping"]

#             # Update minimum null prediction.
#             cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
#             feature_null_score = start_logits[cls_index] + end_logits[cls_index]
#             if min_null_score is None or min_null_score < feature_null_score:
#                 min_null_score = feature_null_score

#             # Go through all possibilities for the `n_best_size` greater start and end logits.
#             start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
#             end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
#             for start_index in start_indexes:
#                 for end_index in end_indexes:
#                     # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
#                     # to part of the input_ids that are not in the context.
#                     if (
#                         start_index >= len(offset_mapping)
#                         or end_index >= len(offset_mapping)
#                         or offset_mapping[start_index] is None
#                         or offset_mapping[end_index] is None
#                     ):
#                         continue
#                     # Don't consider answers with a length that is either < 0 or > max_answer_length.
#                     if end_index < start_index or end_index - start_index + 1 > max_answer_length:
#                         continue

#                     start_char = offset_mapping[start_index][0]
#                     end_char = offset_mapping[end_index][1]
#                     valid_answers.append(
#                         {
#                             "score": start_logits[start_index] + end_logits[end_index],
#                             "text": context[start_char: end_char]
#                         }
#                     )

#         if len(valid_answers) > 0:
#             best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
#         else:
#             # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
#             # failure.
#             best_answer = {"text": "", "score": 0.0}

#         # Let's pick our final answer: the best one or the null answer (only for squad_v2)
#         if not impossible_answer:
#             predictions[example["id"]] = best_answer["text"]
#         else:
#             answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
#             predictions[example["id"]] = answer

#     return predictions

In [ ]:
# final_predictions = postprocess_qa_predictions(small_eval_set, validation_features, raw_predictions.predictions)

In [ ]:
# def print_answer(id):
#     text = [i for i in datasets['validation'] if i['id'] == id][0]
#     print(f"Text: {text['context']}")
#     print(f"Question: {text['question']}")
#     print(f"Answer: {final_predictions[id]}")

In [ ]:
# print_answer('5ad39d53604f3c001a3fe8d1')

In [ ]:
# metric = load_metric("squad_v2" if impossible_answer else "squad")

In [ ]:
# formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
# references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
# metric.compute(predictions=formatted_predictions, references=references)

## Using the Model with 🤗 Pipeline

In [ ]:
from transformers import pipeline

model_checkpoint = "boimbukanbaim/distilbert-indonesian-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
Dalam matematika, turunan atau derivatif dari sebuah fungsi adalah cara mengukur sensitivitas perubahan nilai fungsi terhadap perubahan pada nilai variabelnya. Sebagai contoh, turunan dari posisi sebuah benda bergerak terhadap waktu mengukur kecepatan benda bergerak ketika waktu berjalan. Turunan adalah alat penting dalam kalkulus.

Turunan sebuah fungsi satu variabel di suatu titik, jika itu ada, adalah kemiringan dari garis singgung dari grafik fungsi di titik tersebut. Garis singgung adalah hampiran (aproksimasi) linear terbaik dari fungsi di sekitar titik tersebut. Konsep turunan dapat diperumum untuk fungsi multivariabel. Dalam perumuman ini, turunan dianggap sebagai transformasi linear, dengan translasi yang sesuai, menghasilkan hampiran linear dari grafik fungsi multivariabel tersebut. Matriks Jacobi adalah matriks yang merepresentasikan transformasi linear terhadap suatu basis yang ditentukan. Matriks ini dapat ditentukan dengan turunan parsial dari variabel-variabel independen. Pada fungsi multivariabel bernilai real, matriks Jacobi tereduksi menjadi vektor gradien.

Proses menemukan turunan disebut diferensiasi. Kebalikan proses ini disebut dengan antiturunan. Teorema fundamental kalkulus menyatakan hubungan diferensiasi dengan integrasi. Turunan dan integral adalah dua operasi dasar dalam kalkulus satu-variabel.

Konsep turunan fungsi yang universal banyak digunakan dalam berbagai cabang matematika maupun bidang ilmu yang lain. Dalam bidang ekonomi, turunan digunakan untuk menghitung biaya marginal, total penerimaan, dan biaya produksi. Bidang biologi menggunakan turunan untuk menghitung laju pertumbuhan mikroorganisme, dalam bidang fisika untuk menghitung kepadatan kawat, dalam bidang kimia untuk menghitung laju pemisahan, dalam bidang geografi untuk menghitung laju pertumbuhan penduduk, dan masih banyak lagi.
"""
question = "Apa itu derivatif?"
question_answerer(question=question, context=context)